# Modules

In [1]:
from lib import wrappers
from lib import dqn_model
import time
import numpy as np
import collections
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from tensorboardX import SummaryWriter

C:\Users\james\AppData\Roaming\Python\Python39\site-packages\ale_py\roms\__init__.py:89: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  ROMS = resolve_roms()


# Constants

In [17]:
DEFAULT_ENV_NAME = "BreakoutNoFrameskip-v4"
MEAN_REWARD_BOUND = 19.5
EPSILON_DECAY_LAST_FRAME = 10**5
EPSILON_START = 1.0
EPSILON_FINAL = 0.02
GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 10000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000
REPLAY_START_SIZE = 10000

# Classes

Experienced Buffer

In [18]:
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])

class ExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)



Agent

In [19]:
class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0

    def play_step(self, net, epsilon=0.0, device="cpu"):
        done_reward = None

        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        # do step in the environment
        new_state, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward

# Loss Calculations

In [20]:
def calc_loss(batch, net, tgt_net, device="cpu"):
    states, actions, rewards, dones, next_states = batch

    states_v = torch.tensor(states).to(device)
    next_states_v = torch.tensor(next_states).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.ByteTensor(dones).to(device)

    actions_v2 = actions_v.unsqueeze(-1).long()

    state_action_values = net(states_v).gather(1, actions_v2).squeeze(-1)
    
    next_state_values = tgt_net(next_states_v).max(1)[0]
    next_state_values[done_mask] = 0.0
    next_state_values = next_state_values.detach()

    expected_state_action_values = next_state_values * GAMMA + rewards_v
    return nn.MSELoss()(state_action_values, expected_state_action_values)

# Main

In [22]:

# device = torch.device("gpu")
device = torch.device("cpu")

env = wrappers.make_env(DEFAULT_ENV_NAME)

net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
writer = SummaryWriter(comment="-" + DEFAULT_ENV_NAME)
print(net)

buffer = ExperienceBuffer(REPLAY_SIZE)
agent = Agent(env, buffer)
epsilon = EPSILON_START

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_mean_reward = None

NUM_GAMES = 4000

# for plots
mean_rewards = []
num_games = []
epsilons = []
frames = []


while len(total_rewards) < NUM_GAMES:
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_DECAY_LAST_FRAME)
    if frame_idx % 100 == 0:
        epsilons.append(epsilon)
        frames.append(frame_idx)

    reward = agent.play_step(net, epsilon, device=device)
    if reward is not None and reward > 0:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        ts_frame = frame_idx
        ts = time.time()
        mean_reward = np.mean(total_rewards[-100:])
        mean_rewards.append(mean_reward)
        num_games.append(len(total_rewards))
        print("%d frames over %d games, mean reward %.3f, eps %.2f, speed %.2f f/s" % (
            frame_idx, len(total_rewards), mean_reward, epsilon,
            speed
        ))
        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", mean_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)
        if best_mean_reward is None or best_mean_reward < mean_reward:
            torch.save(net.state_dict(), DEFAULT_ENV_NAME + "-best.dat")
            if best_mean_reward is not None:
                print("Best mean reward updated %.3f -> %.3f, model saved" % (best_mean_reward, mean_reward))
            best_mean_reward = mean_reward
        if mean_reward > MEAN_REWARD_BOUND:
            print("Solved in %d frames!" % frame_idx)
            break

    if len(buffer) < REPLAY_START_SIZE:
        continue

    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict())

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss_t = calc_loss(batch, net, tgt_net, device='cpu')
    loss_t.backward()
    optimizer.step()

writer.close()

plt.plot(num_games, mean_rewards)
plt.xlabel('Number of games played')
plt.ylabel('Mean reward')
plt.show()

plt.plot(range(1, len(total_rewards)+1), total_rewards)
plt.xlabel('Number of games played')
plt.ylabel('Total reward')
plt.show()

plt.hist(total_rewards, bins=30)
plt.xlabel('Reward')
plt.ylabel('Count')
plt.show()

plt.plot(frames, epsilons)
plt.xlabel('Number of frames played')
plt.ylabel('Epsilon')
plt.show()


DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=4, bias=True)
  )
)
198 frames over 1 games, mean reward 2.000, eps 1.00, speed 1069.33 f/s
355 frames over 2 games, mean reward 1.500, eps 1.00, speed 1058.78 f/s
529 frames over 3 games, mean reward 1.333, eps 0.99, speed 996.34 f/s
969 frames over 4 games, mean reward 1.250, eps 0.99, speed 870.41 f/s
1318 frames over 5 games, mean reward 1.200, eps 0.99, speed 872.36 f/s
1570 frames over 6 games, mean reward 1.500, eps 0.98, speed 878.58 f/s
2062 frames over 7 games, mean reward 1.714, eps 0.98, speed 900.55 f/s
2506 frames over 8 games, mean reward 1.625, eps 0.97, speed 882.79 f/s
2715 f

C:\Users\james\AppData\Local\Temp\ipykernel_13536\98137925.py:15: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1773.)
  next_state_values[done_mask] = 0.0


10089 frames over 36 games, mean reward 1.667, eps 0.90, speed 102.23 f/s
10286 frames over 37 games, mean reward 1.676, eps 0.90, speed 35.16 f/s
10726 frames over 38 games, mean reward 1.658, eps 0.89, speed 34.77 f/s
11285 frames over 39 games, mean reward 1.641, eps 0.89, speed 32.72 f/s
11484 frames over 40 games, mean reward 1.650, eps 0.89, speed 31.90 f/s
11648 frames over 41 games, mean reward 1.634, eps 0.88, speed 32.51 f/s
11821 frames over 42 games, mean reward 1.619, eps 0.88, speed 32.48 f/s
12425 frames over 43 games, mean reward 1.628, eps 0.88, speed 33.21 f/s
12658 frames over 44 games, mean reward 1.659, eps 0.87, speed 32.69 f/s
12923 frames over 45 games, mean reward 1.711, eps 0.87, speed 32.46 f/s
13122 frames over 46 games, mean reward 1.717, eps 0.87, speed 32.43 f/s
13325 frames over 47 games, mean reward 1.723, eps 0.87, speed 32.22 f/s
13507 frames over 48 games, mean reward 1.729, eps 0.86, speed 31.85 f/s
13728 frames over 49 games, mean reward 1.735, eps

KeyboardInterrupt: 

EasyProcessError: start error <EasyProcess cmd_param=['Xvfb', '-help'] cmd=['Xvfb', '-help'] oserror=[WinError 2] The system cannot find the file specified return_code=None stdout="None" stderr="None" timeout_happened=False>